In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini text - Single-turn

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [4]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash" # @param {type:"string"}

### Import libraries

In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

### Initalize Vertex AI

In [6]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function

In [7]:
def generate(query:str,
             stream:bool) -> GenerationResponse:

    """
    Generate a response from the model.

    Args:
      query : query string to generate response
      stream : whether to stream the response

    Returns:
      GenerationResponse: The generated response.

    """

    # Request body
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]
    # Google Search grounding.
    tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
        tools=[tool]
    )

    return responses


### Execution

In [9]:
from time import perf_counter

t1_start = perf_counter()

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        아래 내요

        1. 최신 생성형 AI 트랜드에 대해서 설명해주세요.
        2. 올해 12월에 생성형 AI 주요 뉴스에 대해서 10가지 항목으로 요약해주세요.
        3. 생성형 AI의 2025년도 중요한 주제는 어떤것인가요 ?
"""

stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")


Token Count : total_tokens: 125
total_billable_characters: 141


 Usage metadata:
{'prompt_token_count': 125, 'candidates_token_count': 1662, 'total_token_count': 1787}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.103515625
severity: HARM_SEVERITY_LOW
severity_score: 0.32421875
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0284423828
severity: HARM_SEVERITY_LOW
severity_score: 0.275390625
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.197265625
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.154296875
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0966796875
severity: HARM_SEVERITY_MEDIUM
severity_score: 0.5234375
]


## 생성형 AI의 장단점과 2025년 트렌드 분석

### 생성형 AI를 사용하는 것의 장단점:

**장점:**

* **생산성 향상:** 생성형 AI는 다양한 작업을 자동화하여 생산성을 향상시킵니다. 예를 들어, 글쓰기, 이미지 생성, 번역, 코드 작성 등을 더 빠르고 효율적으로 수행할 수 있습니다. 
* **창의성 증진:** 예술, 디자인, 콘텐츠 제작 등 창의적인 분야에서 새로운 아이디어와 표현 방식을 제공하여 창의성을 증진시킵니다.
* **개인화된 경험 제공:** 사용자의 선호도와 요구사항에 맞춰 개인화된 콘텐츠, 서비스, 제품을 제공할 수 있습니다.
* **새로운 비즈니스 모델 창출:** 기존 비즈니스 모델을 혁신하고 새로운 비즈니스 기회를 창출하는 데 기여합니다.

**단점:**

* **정보의 정확성 문제:** 생성형 AI는 학습 데이터에 의존하기 때문에 잘못된 정보를 생성할 수 있습니다. 이는 헐루시네이션(hallucination) 문제로 불리며, 사용자가 정보의 정확성을 주의해야 합니다.
* **편향성 문제:** 학습 데이터의 편향성으로 인해 생성된 결과물에도 편향성이 반영될 수 있습니다. 이는 윤리적 문제를 야기할 수 있으며, AI 개발 과정에서 편향성을 최소화하기 위한 노력이 필요합니다.
* **저작권 문제:** 생성형 AI가 생성한 콘텐츠의 저작권 문제는 아직 명확하지 않습니다. AI가 생성한 콘텐츠의 소유권, 저작권 침해 여부 등에 대한 법적 규정이 미비한 상황입니다.
* **일자리 감소 우려:** 일부 작업이 자동화되면서 일자리 감소를 야기할 수 있습니다. 새로운 일자리가 창출되겠지만, 기존 직업의 변화와 새로운 기술 교육이 필요합니다.

### 최신 생성형 AI 트렌드:

* **온디바이스 AI:** 스마트폰, 냉장고, TV 등 개인 기기에 AI 기능을 탑재하여 네트워크 연결 없이도 AI 서비스를 이용할 수 있도록 하는 기술입니다. 개인 정보 보호와 보안 강화에 유리합니다.
* **멀티모달 AI:** 텍스트, 이미지, 음성, 비디오 등 다양한 형태의 데이터를 이해하고 처리하는 AI 기술입니다. 더 풍부하고 복잡한 작업을 수행할 수 있습니다. 
* **AI 에이전트:** 사용자의 명령을 수행하고, 스스로 학습하며, 다양한 작업을 자동화하는 AI 시스템입니다. 사용자 편의성을 향상시키고, 더 효율적인 작업 환경을 제공합니다.
* **추론형 AI:** 생성된 결과물의 근거를 제공하고, 논리적 추론 능력을 향상시키는 AI 기술입니다. 정보의 신뢰도를 높이고, 더 정확한 결과를 얻을 수 있습니다.
* **AI 윤리 및 규제:** 생성형 AI의 사회적 영향을 고려하여 윤리적 규범과 규제를 강화하는 추세입니다. AI의 편향성, 책임 소재, 저작권 문제 등을 해결하기 위한 노력이 필요합니다.


### 2025년 주요 생성형 AI 뉴스 10가지:

1. **AI 에이전트의 상용화:** 다양한 분야에서 AI 에이전트가 상용화되어 사용자 경험을 혁신하고 생산성을 향상시킵니다.
2. **추론형 AI의 발전:** AI 모델의 논리적 추론 능력이 향상되어 더 정확하고 신뢰할 수 있는 결과를 제공합니다.
3. **AI 기반 사이버 보안 강화:** AI를 활용하여 더욱 정교한 사이버 공격을 방어하고, 사이버 보안 시스템을 강화합니다.
4. **AI 칩의 발전:** AI 연산에 특화된 고성능 저전력 AI 칩이 개발되어, 온디바이스 AI 기술 발전을 가속화합니다.
5. **AI 윤리 규제 강화:** AI 개발 및 사용에 대한 윤리적 규정과 규제가 강화되어, AI 기술의 사회적 책임을 강조합니다.
6. **산업별 특화 AI 솔루션:** 각 산업의 특성에 맞는 AI 솔루션이 개발되어, 비즈니스 효율성과 경쟁력을 향상시킵니다.
7. **AI 기반 스마트홈 확대:** AI 기술을 탑재한 스마트홈 기기와 서비스가 확산되어, 편리하고 개인화된 주거 환경을 제공합니다.
8. **AI 기반 의료 서비스 발전:** AI 기술을 활용한 질병 진단, 치료, 예방 서비스가 발전하여, 의료 서비스 질을 향상시킵니다.
9. **AI 기반 교육 시스템 변화:** AI 기술을 활용한 개인 맞춤형 교육, 스마트 러닝 시스템이 확산되어, 교육 효과를 극대화합니다.
10. **AI 기반 예술 활동 확대:** AI를 활용한 음악, 미술, 문학 작품 제작이 활발해지면서, 예술 분야에 새로운 창조적 가능성을 제시합니다.

### 2025년 중요한 생성형 AI 주제:

* **AI 에이전트의 확산:** 개별 작업을 수행하는 도구를 넘어, 사용자와 상호작용하고 스스로 학습하며 다양한 작업을 수행하는 AI 에이전트가 어떻게 발전하고, 사회에 어떤 영향을 미칠지 주목해야 합니다.
* **AI의 신뢰성 및 윤리:**  생성형 AI가 생성하는 정보의 정확성과 편향성 문제를 해결하고, AI 기술의 윤리적 사용을 보장하기 위한 노력이 중요합니다. AI 관련 법적 규제와 사회적 합의가 필요합니다.
* **AI와 인간의 협업:** AI는 인간의 능력을 보완하고, 새로운 가치를 창출하는 협력 파트너 역할을 할 수 있습니다. AI 기술과 인간의 협력 방식을 어떻게 발전시킬지 고민해야 합니다.
* **AI의 경제적 영향:**   AI 기술의 발전은 생산성 향상, 새로운 산업 창출 등 경제적 성장을 촉진할 수 있습니다. 하지만 일자리 감소 등 사회적 부작용을 최소화하기 위한 정책과 노력이 필요합니다. 

**결론:**  생성형 AI는 빠르게 발전하고 있으며, 우리 사회에 큰 영향을 미칠 것입니다.  AI 기술의 장점을 활용하고, 단점을 최소화하기 위한 지속적인 노력과 사회적 합의가 필요합니다.


Time : 9.421124446000022 seconds


